## Introduction

The Bank Customer sample data is an integrated dataset of 1) accounts and account owners, 2) purchases by the people who own those accounts, and 3) transactions between these accounts. The combined dataset is then used to perform the following common graph query and analyses: pattern matching, detection of cycles, finding important nodes, community detection, and recommendation.

![](https://user-images.githubusercontent.com/4862919/63682754-d46f0680-c822-11e9-89ae-9c84e0aec2b3.jpg)

### Connect to Graph Server

In [1]:
import json
import os
import platform
import sys
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pypgx as pgx

base_url = "http://graph-server:7007"
username = "graph_dev"
password = "Welcome1"

def generateToken():
    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)
    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')
    except HTTPError as err:
        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))
        os._exit(1)
 
session = pgx.get_session(base_url=base_url, token=generateToken())
print(session)

PgxSession(id: d104dc26-6f13-435e-9b58-833623f04be9, name: python_pgx_client)


### Get Pre-loaded Graph

In [2]:
graph = session.get_graph("Customer 360")
graph

PgxGraph(name: Customer 360, v: 15, e: 24, directed: True, memory(Mb): 0)

In [3]:
graph.query_pgql("""
  SELECT ID(a), a.account_no, a.balance
  FROM MATCH (a)
  WHERE a.type = 'account'
    AND a.account_no = 'xxx-yyy-201'
""").print()

+-------------------------------+
| ID(a) | account_no  | balance |
+-------------------------------+
| 201   | xxx-yyy-201 | 1500    |
+-------------------------------+



### One-hop Relationships from an Account

Show any related information about the account "xxx-yyy-202"

In [4]:
graph.query_pgql("""
  SELECT a.account_no, LABEL(e), n.type
  FROM MATCH (a)-[e]-(n)
  WHERE a.type = 'account'
    AND a.account_no = 'xxx-yyy-202'
  ORDER BY LABEL(e)
""").print()

+------------------------------------+
| account_no  | LABEL(e)  | type     |
+------------------------------------+
| xxx-yyy-202 | owned_by  | customer |
| xxx-yyy-202 | purchased | merchant |
| xxx-yyy-202 | purchased | merchant |
| xxx-yyy-202 | purchased | merchant |
| xxx-yyy-202 | transfer  | account  |
| xxx-yyy-202 | transfer  | account  |
| xxx-yyy-202 | transfer  | account  |
| xxx-yyy-202 | transfer  | account  |
| xxx-yyy-202 | transfer  | account  |
+------------------------------------+



### Pattern Matching

Find accounts that had an inbound and an outbound transfer, of over 500, on the same day. The PGQL query for this is:

In [5]:
graph.query_pgql("""
  SELECT a.account_no
       , a.balance
       , t1.amount AS t1_amount
       , t2.amount AS t2_amount
       , t1.date
  FROM MATCH (a) <-[t1:transfer]- (a1)
     , MATCH (a) -[t2:transfer]-> (a2)
  WHERE t1.date = t2.date
    AND t1.amount > 500
    AND t2.amount > 500
    AND a.balance < 300
""").print()

+------------------------------------------------------------+
| account_no  | balance | t1_amount | t2_amount | date       |
+------------------------------------------------------------+
| xxx-yyy-202 | 200     | 900       | 850       | 2018-10-06 |
+------------------------------------------------------------+



### Detect Cycles

Next we use PGQL to find a series of transfers that start and end at the same account such as A to B to A, or A to B to C to A.

![](https://user-images.githubusercontent.com/4862919/63682879-1ef08300-c823-11e9-8559-a142851b1ed1.jpg)
![](https://user-images.githubusercontent.com/4862919/63683037-8a3a5500-c823-11e9-9677-c3788f4df1cd.jpg)

In [6]:
graph2 = graph.filter(pgx.EdgeFilter("edge.label()='transfer'"))
graph2

PgxGraph(name: sub-graph_17, v: 6, e: 8, directed: True, memory(Mb): 0)

In [7]:
analyst = session.create_analyst()
result = analyst.pagerank(graph2);
result

VertexProperty(name: pagerank, type: double, graph: sub-graph_17)

In [8]:
graph2.query_pgql("""
  SELECT a.account_no, a.pagerank
  MATCH (a)
  ORDER BY a.pagerank DESC
""").print()

+-------------------------------------+
| a.account_no | a.pagerank           |
+-------------------------------------+
| xxx-yyy-201  | 0.18012007557258927  |
| xxx-yyy-204  | 0.1412461615467829   |
| xxx-yyy-203  | 0.1365633635065475   |
| xxx-yyy-202  | 0.12293884324085073  |
| xxx-zzz-002  | 0.05987452026569676  |
| xxx-zzz-001  | 0.025000000000000005 |
+-------------------------------------+



## Community Detection

Let’s find which subsets of accounts form communities. That is, there are more transfers among accounts in the same subset than there are between those and accounts in another subset. We’ll use the built-in weekly and strongly connected components algorithm.

The first step is to create a subgraph that only has the accounts and the transfers among them. This is done by creating and applying an edge filter (for edges with the table "transfer") to the graph.

![](https://user-images.githubusercontent.com/4862919/63683125-c2419800-c823-11e9-8ca1-124a50398a5e.jpg)

### Weakly Connected Component

In [9]:
graph2.destroy_vertex_property_if_exists("wcc")
analyst.wcc(graph2)

PgxPartition(graph: sub-graph_17, components: 1)

In [10]:
graph2.query_pgql("""
  SELECT a.wcc AS component_id, COUNT(*) AS count
  FROM MATCH (a)
  GROUP BY a.wcc
  ORDER BY a.wcc
""").print()

+----------------------+
| component_id | count |
+----------------------+
| 0            | 6     |
+----------------------+



### Strongly Connected Component

In [11]:
graph2.destroy_vertex_property_if_exists("scc_kosaraju")
analyst.scc_kosaraju(graph2)

PgxPartition(graph: sub-graph_17, components: 3)

In [12]:
graph2.query_pgql("""
  SELECT a.scc_kosaraju AS component_id, COUNT(a.account_no) AS count
  FROM MATCH (a)
  GROUP BY a.scc_kosaraju
  ORDER BY a.scc_kosaraju
""").print()

+----------------------+
| component_id | count |
+----------------------+
| 0            | 1     |
| 1            | 4     |
| 2            | 1     |
+----------------------+



## Recommendation

Lastly let's use Personalized PageRank to find stores that John may purchase from given that people he is connected to have made purchases from those stores.

![](https://user-images.githubusercontent.com/4862919/63682622-8f4ad480-c822-11e9-9979-b8a1525240fe.jpg)

In [13]:
graph3 = graph.filter(pgx.EdgeFilter("edge.label()='purchased'"))
graph3

PgxGraph(name: sub-graph_21, v: 9, e: 11, directed: True, memory(Mb): 0)

### Reverse Edges

![](https://user-images.githubusercontent.com/4862919/63683184-eef5af80-c823-11e9-85be-495295d0c2a3.jpg)
![](https://user-images.githubusercontent.com/4862919/63683190-f1f0a000-c823-11e9-8e3d-96152f4edf98.jpg)`

In [14]:
graph4 = graph3.clone_and_execute_pgql("""
  INSERT EDGE e BETWEEN m AND a LABELS ("purchased_by")
  FROM MATCH (a)->(m)
""")
graph4

PgxGraph(name: sub-graph_22, v: 9, e: 22, directed: True, memory(Mb): 0)

In [15]:
graph4.query_pgql("""
  SELECT x.name, LABEL(r), a.account_no
  FROM MATCH (a)<-[r:purchased_by]-(x)
""").print()

+-------------------------------------------+
| name         | LABEL(r)     | account_no  |
+-------------------------------------------+
| Apple Store  | purchased_by | xxx-yyy-201 |
| PC Paradise  | purchased_by | xxx-yyy-201 |
| Apple Store  | purchased_by | xxx-yyy-202 |
| PC Paradise  | purchased_by | xxx-yyy-202 |
| Asia Books   | purchased_by | xxx-yyy-202 |
| Apple Store  | purchased_by | xxx-yyy-203 |
| PC Paradise  | purchased_by | xxx-yyy-203 |
| Kindle Store | purchased_by | xxx-yyy-203 |
| Kindle Store | purchased_by | xxx-yyy-204 |
| Asia Books   | purchased_by | xxx-yyy-204 |
| ABC Travel   | purchased_by | xxx-yyy-204 |
+-------------------------------------------+



### Personalized PageRank

In [16]:
rs = graph4.query_pgql("SELECT ID(a) FROM MATCH (a) WHERE a.account_no = 'xxx-yyy-201'")
vertex = graph4.get_vertex(rs.get_row(0))
graph4.destroy_vertex_property_if_exists("ppr")
analyst.personalized_pagerank(graph4, vertex, rank="ppr")

VertexProperty(name: ppr, type: double, graph: sub-graph_22)

### Recommended Products

In [17]:
graph4.query_pgql("""
  SELECT ID(x), x.name, x.ppr
  FROM MATCH (x)
  WHERE x.type = 'merchant'
    AND NOT EXISTS (
      SELECT *
      FROM MATCH (x)-[:purchased_by]->(a)
      WHERE a.account_no = 'xxx-yyy-201'
    )
  ORDER BY x.ppr DESC
""").print()

+--------------------------------------------+
| ID(x) | name         | ppr                 |
+--------------------------------------------+
| 303   | Kindle Store | 0.04932640133302745 |
| 304   | Asia Books   | 0.04932640133302745 |
| 305   | ABC Travel   | 0.01565535511504672 |
+--------------------------------------------+



## Load from Database (Appendix)

In [2]:
statement = '''
CREATE PROPERTY GRAPH "Customer 360 DB"
  VERTEX TABLES (
    customer_360.customer
      LABEL "Customer"
  , customer_360.account
      LABEL "Account"
  , customer_360.merchant
      LABEL "Merchant"
  )
  EDGE TABLES (
    customer_360.owned_by
      SOURCE KEY(from_id) REFERENCES account
      DESTINATION KEY(to_id) REFERENCES customer
      LABEL "owned_by"
  , customer_360.parent_of
      SOURCE KEY(from_id) REFERENCES customer
      DESTINATION KEY(to_id) REFERENCES customer
      LABEL "parent_of"
  , customer_360.purchased
      SOURCE KEY(from_id) REFERENCES account
      DESTINATION KEY(to_id) REFERENCES merchant
      LABEL "purchased"
  , customer_360.transfer
      SOURCE KEY(from_id) REFERENCES account
      DESTINATION KEY(to_id) REFERENCES account
      LABEL "transfer"
  )
'''

In [3]:
session.prepare_pgql(statement).execute()

False

In [4]:
graph = session.get_graph("Customer 360 DB")

In [5]:
graph

PgxGraph(name: Customer 360 DB, v: 15, e: 24, directed: True, memory(Mb): 0)